## В этом домашнем задании вы сделаете первые шаги в мире линейной бинарной классификации!

In [4]:
import pandas as pd
import numpy as np
import warnings

#### Задание 1

Мы будем работать с данными **Microsoft Malware Detection**

Таргетом будет последний столбец `HasDetection`, который принимает значения $\{0,\, 1\}$ в случае отсутствия или наличия вируса на компьютере соответственно. Признаками будут выступать всевозможные характеристики компьютера.

In [5]:
data = pd.read_csv('train.csv')

/var/folders/lp/4tdqkmmx5j5bmzdwcctlrcfw0000gn/T/ipykernel_43037/40742158.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('train.csv')


In [14]:
data.shape

(200000, 81)

Удалите константные признаки и признаки `ProductName` `MachineIdentifier`

In [6]:
### Your code is here

data = data.drop(columns=["ProductName", "MachineIdentifier"], axis=1)






In [10]:
def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlation(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels = labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print(get_top_abs_correlation(data[numeric_columns], n=15))

KeyError: ('Census_ActivationChannel', 'Census_OEMModelIdentifier')

Посмотрите на соотношение классов в таргете. Все ли хорошо?

In [7]:
print(sum(data['HasDetections'] == 1), '- positive class,')
print(sum(data['HasDetections'] == 0), '- negative class')

100060 - positive class,
99940 - negative class


Ответьте на вопрос: почему с вашей точки зрения важно иметь представление о балансе классов в ваших данных?

Избавьтесь от пропусков в данных! 

Новый для нас прием: признаки с более чем половиной пропусков следует удалить.

Согласитесь, если в вашей колонке среди 100 объектов всего лишь у 2 есть какое-то непропущенное значение, странно все остальные заполнять средним от этих двух чисел. Такие "редкие" признаки лучше вообще опустить!


В категориальных колонках заменим отсутствующую категорию просто некоторой новой и назовем ее `NaN`.

А в числовых, ради разнообразия, заполним пропуски медианным значением.

In [8]:
### Your code is here
data = data.drop(columns=["DefaultBrowsersIdentifier", "PuaMode", "Census_ProcessorClass", "Census_InternalBatteryType","Census_IsFlightingInternal","Census_ThresholdOptIn", "Census_IsWIMBootEnabled", "IsBeta"], axis=1)




In [49]:
data.isna().sum()

EngineVersion                              0
AppVersion                                 0
AvSigVersion                               0
RtpStateBitfield                           0
IsSxsPassiveMode                           0
                                          ..
Census_IsPenCapable                        0
Census_IsAlwaysOnAlwaysConnectedCapable    0
Wdft_IsGamer                               0
Wdft_RegionIdentifier                      0
HasDetections                              0
Length: 73, dtype: int64

In [9]:
df_numeric_col = data.loc[:,data.dtypes!=np.object].columns
df_categorical_col = data.loc[:,data.dtypes==np.object].columns

/var/folders/lp/4tdqkmmx5j5bmzdwcctlrcfw0000gn/T/ipykernel_43037/4175800921.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_numeric_col = data.loc[:,data.dtypes!=np.object].columns
/var/folders/lp/4tdqkmmx5j5bmzdwcctlrcfw0000gn/T/ipykernel_43037/4175800921.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_categorical_col = data.loc[:,data.dtypes==np.object].columns


In [10]:
data[df_categorical_col] = data[df_categorical_col].fillna('NaN', axis=0)

In [11]:
for col in df_numeric_col:
    median = data.loc[:, col].median()
    data.loc[:, col].fillna(median, inplace=True)

Создайте копию полученного датафрейма и положите ее в переменную data_2. Понадобится в следующих заданиях.

In [12]:
data_2 = data.copy()

Так же поработаем над всеми категориальными колонкам перед запуском непосредственно моделей.

Провернем самый базовый и наглый метод - несмотря на количество уникальных значений в каждой категории, просто применим ко всей категориальной части датасета `OneHotEncoding`

In [13]:
### Your code is here
for col in df_categorical_col:
    onehot = pd.get_dummies(data[col], prefix=col, drop_first=True)
    data = pd.concat((data.drop(col, axis=1), onehot), axis=1)


In [14]:
data.shape

(200000, 6114)

###  Разделим выборку на тренировочную и тестовую

P.S. в задачах классификации, как и в задаче регрессии, можно использовать технологию Кросс-Валидации. Например, по одному из двух следующих сценариев:

1) Отделить валидацию и тест, произвести подбор лучшей модели с помощью `K-Fold` на валидации, финально обучить выбранную модель на всей валидации и замерить качество на заранее отложенном финальном тесте!

2) Всю выборку назвать валидационной и на ней применить `K-Fold` без финального замера.

В этой домашней работе попросим Вас быть еще проще! :)
Реализуем просто технологию отложенной выборки в пропорции 3:1

In [15]:
from sklearn.model_selection import train_test_split 

X = data.drop(columns=['HasDetections'])
y = data['HasDetections']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=1)

Соберите `Pipeline`, реализовав в нем 2 шага: стандартизация данных через `StandardScaler` и обучение логистической регрессии с помощью `LogisticRegression`, положите результаты в переменную `pipe`, а в классе модели `LogisticRegression` укажите параметр `penalty='none'`

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

### Your code is here

pipe = Pipeline([('scaler', StandardScaler()), ('LR', LogisticRegression(penalty='none'))])

#pipe.fit(X_train, y_train)

Чтобы замерить качество работы такой модели на трейне и на тесте воспользуемся функцией `cross_validate`

Вопрос: что передавать ей в параметр cv? Ведь мы уже разделили нашу выборку на трейн и тест (имеем всего 1 fold). Для этого можно просто передать список от кортежа, содержащего индексы тренировочных и тестовых объектов.

In [17]:
from sklearn.model_selection import cross_validate
import datetime

custom_cv = [(X_train.index.to_list(), X_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X, y, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)


print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")

/Users/Mamster/PycharmProjects/machinelearning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy на трейне: 0.651
Accuracy на тесте: 0.623
Время работы алгоритма: 0:08:09.777125


Что можете сказать про время работы алгоритма?

Очевидно, оно достаточно большие. Уж тем более для линейных моделей.

Такое как раз-таки происходит из-за того, что количество фичей, который мы передали нашей модели - гигантское! Классу требуется много времени и памяти, чтобы обработать датасет.

Поэтому те колонки, в которых количество уникальных категорий превышает какое-то адекватное число, следует кодировать иначе, нежели с помощью технологии `One-Hot-Encoding`.

Теперь вы верите, что более умные кодировки зачастую прям необходимы! Раньше мы этот факт не демонстрировали!

Дело еще вот в чем: в классе `LogisticRegression`, как и, например, `Lasso`, есть параметр, ограничивающий максимальное количество итераций во время обучения модели. Так, если данных много и итераций тоже ожидается большое число, найденная разделяющая гиперплоскость может оказаться не самой лучшей, так как наш алгоритм (будь то градиентный спуск или любой иной) просто 'не доползет'. 

#### Задание 2

Теперь попробуем другой метод кодирования категориальных колонок, а именно счётчики.
Построем ту же модель и на том же разделении, просто заново иначе переобработаем датасет. 

Для тех категориальных признаков, у которых количество уникальных значений в колоночках больше 5, применим `MeanTargetEncoding`.

Для остальных оставим любимый `OneHotEncoding` (как делали на практике и в предыдущем уроке).

In [18]:
### Your code is here
for col in df_categorical_col:

    if data_2[col].nunique() < 5:
        onehot = pd.get_dummies(data_2[col], prefix=col, drop_first=True)
        data_2 = pd.concat((data_2.drop(col, axis=1), onehot), axis=1)

    else:
        mean_target = data_2.groupby(col)["HasDetections"].mean()
        data_2[col] = data_2[col].map(mean_target)


In [54]:
data_2.head()

,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,...,Platform_windows7,Platform_windows8,Processor_x64,Processor_x86,Census_DeviceFamily_Windows.Server,Census_OSArchitecture_arm64,Census_OSArchitecture_x86,Census_GenuineStateName_IS_GENUINE,Census_GenuineStateName_OFFLINE,Census_GenuineStateName_UNKNOWN
0,0.552153,0.530941,0.587952,7.0,0,53447.0,1.0,1.0,1,29,...,0,0,1,0,0,0,0,1,0,0
1,0.448751,0.528931,0.484434,7.0,0,53447.0,1.0,1.0,1,93,...,0,0,1,0,0,0,0,0,1,0
2,0.552153,0.530941,0.557522,7.0,0,53447.0,1.0,1.0,1,86,...,0,0,1,0,0,0,0,1,0,0
3,0.552153,0.530941,0.714442,7.0,0,53447.0,1.0,1.0,1,88,...,0,0,1,0,0,0,0,1,0,0
4,0.552153,0.530941,0.541395,7.0,0,53447.0,1.0,1.0,1,18,...,0,0,1,0,0,0,0,1,0,0


In [19]:
X_2 = data_2.drop(columns=['HasDetections'])
y_2 = data_2['HasDetections']

X2_train, X2_test, y2_train, y2_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=1)

Опять обучим модель, пока что без изменений! Скажите, стало ли быстрее? А что с качеством?

In [20]:
### Your code is here

custom_cv = [(X2_train.index.to_list(), X2_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X_2, y_2, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)


print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")




Accuracy на трейне: 0.638
Accuracy на тесте: 0.638
Время работы алгоритма: 0:00:03.216917


#### Задание 3: Регуляризация

Как и в моделях регрессии, решая задачу классификации, можем штрафовать минимизируемый функционал за большие веса, добавив к нему L1 или L2 норму весов (все как раньше!).

Для этого в изначальном классе `LogisticRegression` изменить параметр `penalty` на l1 или l2 соответственно. Выберите второй вариант! Можно воспользоваться методом `set_params` и применить его к `pipe`.

In [21]:
### Your code is here

pipe = Pipeline([('scaler', StandardScaler()), ('LR', LogisticRegression(penalty='l2'))])


Теперь наша модель будет строить логистическую регрессию с L2 регуляризатором! Помним, что у регуляризируемых моделей есть гиперпараметр, контролирующий силу регуляризации, который выбирается ДО запуска метода fit, то есть заранее, когда модель еще не обучена. 

Конечно же, выбор этого параметра влияет итоговые результаты. Хочется поперебирать различные параметры регуляризации и найти такой, при котором качество на тесте окажется лучшим! 

Сгенерируем массив гиперпараметра, которые планируем перебрать:

In [22]:
alphas = np.linspace(0.01, 100, 100)
alphas

array([1.000e-02, 1.020e+00, 2.030e+00, 3.040e+00, 4.050e+00, 5.060e+00,
       6.070e+00, 7.080e+00, 8.090e+00, 9.100e+00, 1.011e+01, 1.112e+01,
       1.213e+01, 1.314e+01, 1.415e+01, 1.516e+01, 1.617e+01, 1.718e+01,
       1.819e+01, 1.920e+01, 2.021e+01, 2.122e+01, 2.223e+01, 2.324e+01,
       2.425e+01, 2.526e+01, 2.627e+01, 2.728e+01, 2.829e+01, 2.930e+01,
       3.031e+01, 3.132e+01, 3.233e+01, 3.334e+01, 3.435e+01, 3.536e+01,
       3.637e+01, 3.738e+01, 3.839e+01, 3.940e+01, 4.041e+01, 4.142e+01,
       4.243e+01, 4.344e+01, 4.445e+01, 4.546e+01, 4.647e+01, 4.748e+01,
       4.849e+01, 4.950e+01, 5.051e+01, 5.152e+01, 5.253e+01, 5.354e+01,
       5.455e+01, 5.556e+01, 5.657e+01, 5.758e+01, 5.859e+01, 5.960e+01,
       6.061e+01, 6.162e+01, 6.263e+01, 6.364e+01, 6.465e+01, 6.566e+01,
       6.667e+01, 6.768e+01, 6.869e+01, 6.970e+01, 7.071e+01, 7.172e+01,
       7.273e+01, 7.374e+01, 7.475e+01, 7.576e+01, 7.677e+01, 7.778e+01,
       7.879e+01, 7.980e+01, 8.081e+01, 8.182e+01, 

Чтобы отобрать среди данного массива гиперпараметров лучший, воспользуйтесь конструкцией `GridSearchCV` из `sklearn`

In [26]:
from sklearn.model_selection import GridSearchCV

param_grid ={
    'LR__C': alphas
}

### Your code is here
search = GridSearchCV(pipe, param_grid, cv=custom_cv, scoring='accuracy')

search.fit(X_2, y_2)

print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)

Best parameter (CV score=0.63822):
{'LR__C': 0.01}


#### Задание 4: Бонус

Как вы знаете, подбор признаков является одной из самых важных частей машинного обучения. Сейчас вы попробуете на основе имеющихся признаков сгенерировать новые. В качестве новых признаков будем использовать мономы 2 степени. Можно использовать регуляризацию различного рода, выбор энкодера тоже за вами. Ваша задача - добиться качества `0.65` на тестовой выборке

In [19]:
### Your code is here

